<a href="https://colab.research.google.com/github/henryfuentesbenito/andi2_challenge-/blob/main/andi2_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.0 MB/s eta 0:00:00


In [3]:
import glob
import os
import pandas as pd

def load_trajectory_data_from_folders(main_folder):
    all_trajectory_data = []
    folder_names = os.listdir(main_folder)
    for folder_name in folder_names:
        folder_path = os.path.join(main_folder, folder_name)
        if os.path.isdir(folder_path):
            file_paths = glob.glob(os.path.join(folder_path, '*.csv'))
            for file_path in file_paths:
                trajectory_data = pd.read_csv(file_path)
                all_trajectory_data.append(trajectory_data)
    return all_trajectory_data

# Ruta de la carpeta principal que contiene las subcarpetas con archivos csv
main_folder = '/content/drive/MyDrive/andi2_dataset6/ref/track_2'

# Cargar datos de trayectorias desde las subcarpetas con archivos csv
trajectory_data_list = load_trajectory_data_from_folders(main_folder)

In [4]:
len(trajectory_data_list)

60

In [5]:
trajectory_data_list

[      traj_idx  frame           x          y
 0          0.0  117.0  207.293228  24.012350
 1          0.0  118.0  207.455854  23.816246
 2          0.0  119.0  207.221884  23.649520
 3          0.0  120.0  208.964227  23.341389
 4          0.0  121.0  208.716672  25.648786
 ...        ...    ...         ...        ...
 5640      37.0  195.0  160.510163  94.636454
 5641      37.0  196.0  159.001391  95.050213
 5642      37.0  197.0  157.629891  95.083858
 5643      37.0  198.0  157.493789  96.188235
 5644      37.0  199.0  156.793114  93.436019
 
 [5645 rows x 4 columns],
       traj_idx  frame           x           y
 0          0.0    0.0  185.315175  147.897292
 1          0.0    1.0  186.553251  148.144029
 2          0.0    2.0  182.908574  147.351226
 3          0.0    3.0  185.952860  147.385078
 4          0.0    4.0  186.572022  147.576057
 ...        ...    ...         ...         ...
 4224      30.0  195.0  143.907691   53.086940
 4225      30.0  196.0  144.154966   53.8440

In [33]:
import numpy as np
import torch
from skimage import measure
from torch_geometric.data import Data

class GraphFromTrajectoryData:
    """Graph representation of particle trajectories."""

    def __init__(self, connectivity_radius, max_frame_distance): #radio de conectividad y la distancia máxima entre cuadros, respectivamente.
        """Initialize graph constructor."""
        self.connectivity_radius = connectivity_radius
        self.max_frame_distance = max_frame_distance
        #esta clase proporciona una estructura para construir grafos a partir de datos de trayectorias,
        #con la capacidad de especificar el radio de conectividad y la distancia máxima entre cuadros como parámetros.

    def __call__(self, trajectories):  #se utiliza para construir grafos a partir de las trayectorias proporcionadas.
        """Construct graphs from trajectories."""
        graphs = []

        # Process trajectories group by traj_idx
        for traj_idx, trajectory_group in trajectories.groupby('traj_idx'): #Itera sobre las trayectorias, agrupadas
                                                                          #por el índice de trayectoria (traj_idx).
                                                                          #Esto significa que las trayectorias con el mismo traj_idx se agruparán juntas.
            graph = self.construct_graph(trajectory_group) #Llama al método construct_graph con el grupo de trayectorias actual para construir un grafo a partir de ellas
            graphs.append(graph)

        return graphs

    def construct_graph(self, trajectory_group): #se encarga de construir un grafo a partir de un grupo de trayectorias dadas
        """Construct graph from a group of trajectories.""" #toma un parámetro trajectory_group, que es un grupo de trayectorias que comparten el mismo índice de trayectoria.
        x, edge_index, edge_attr = [], [], []

        # Compute node features
        features, index_labels = self.compute_node_features(trajectory_group) #Calcula las características de los nodos y las etiquetas de índice utilizando el método compute_node_features
        x.extend(features)
        node_indices = np.arange(len(features))

        # Compute connectivity
        edge_index, edge_attr = self.compute_connectivity(features, node_indices) #Calcula los índices de los bordes y los atributos de los bordes utilizando el método compute_connectivity

        x = torch.tensor(x, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float).view(-1, 1)

        # Construct Data object
        graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

        return graph


    def compute_node_features(self, trajectory_group): #se encarga de calcular las características de los nodos de un grafo a partir de un grupo de trayectorias.
      #calcula la media de las coordenadas x e y de un grupo de trayectorias
      """Compute node features."""
      features = trajectory_group[['x', 'y']].mean(axis=0).values # Solo las coordenadas de la trayectoria
      return [features], [trajectory_group['traj_idx'].iloc[0]]


    def compute_connectivity(self, features, node_indices): #calcula la conectividad entre los nodos de un grupo de trayectorias dado
      """Compute connectivity."""
      edge_index = []
      edge_attr = []
      num_nodes = len(features)

      for i in range(num_nodes): # el primer for itera sobre todos los nodos, y el segundo for itera sobre los nodos restantes.
        for j in range(i + 1, num_nodes):
            distance = np.linalg.norm(features[i][:2] - features[j][:2]) #Para cada par de nodos, se calcula la distancia euclidiana entre sus características de posición x e y
            frame_distance = abs(features[i][2] - features[j][2])  # Distancia entre fotogramas
            if distance < self.connectivity_radius and frame_distance <= self.max_frame_distance:
                edge_index.append([node_indices[i], node_indices[j]])
                edge_attr.append(distance)
      #calcula las conexiones entre nodos del grafo si su distancia euclidiana es menor que un radio específico
      #y la diferencia entre sus fotogramas es menor o igual que una distancia máxima especificada

      return edge_index, edge_attr



In [35]:
trajectory_graphs = []

# Aquí va el código para cargar los datos de trayectorias en `trajectory_data_list`

# Luego, se itera sobre cada DataFrame en la lista y construyes los grafos
for trajectory_data in trajectory_data_list:
    # Construir el grafo a partir de los datos de trayectoria
    graph = GraphFromTrajectoryData(connectivity_radius=10, max_frame_distance=2)(trajectory_data)
    trajectory_graphs.append(graph)

In [36]:
len(trajectory_graphs)

60

In [37]:
trajectory_graphs[0]

[Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0], edge_attr=[0, 1]),
 Data(x=[1, 2], edge_index=[0],